Dataset used: COVID-19 Data Lake - https://registry.opendata.aws/aws-covid19-lake/
Tutorial: COVID 19 - Build End to End Data Engineering Project | PART 1 - https://www.youtube.com/watch?v=gFWu-SSzRzc&t=464s

In [1]:
#%pip install --user boto3
#%pip install --user awscli

In [21]:
import boto3
import pandas as pd
import psycopg2 
import json
import os
import configparser
import pprint
from botocore.client import ClientError
import subprocess
import time

In [22]:
config = configparser.ConfigParser()
config.read_file(open('cluster.config'))

In [23]:
AWS_KEY = os.environ.get('AWS_KEY')
AWS_SECRET = os.environ.get('AWS_SECRET')
AWS_REGION = config.get("AWS","AWS_REGION")

S3_BUCKET_NAME = config.get("S3","S3_BUCKET_NAME")
S3_ACL = config.get("S3","S3_ACL")
S3_LOCATION = config.get("S3","S3_LOCATION")

1st Step - Create S3 Bucket using Python

In [24]:
s3 = boto3.client('s3', 
                aws_access_key_id=AWS_KEY,
                aws_secret_access_key=AWS_SECRET,
                region_name=AWS_REGION)

try:
    s3.head_bucket(Bucket=S3_BUCKET_NAME)
    bucket_exists = 'Yes'
    print("The bucket exists")
except ClientError:
    bucket_exists = 'No'
    print("The bucket does no exist or you have no access")

The bucket does no exist or you have no access


In [25]:
try:
    if bucket_exists == 'No':
        create_bucket = s3.create_bucket(Bucket=S3_BUCKET_NAME, 
                    ACL=S3_ACL, 
                    CreateBucketConfiguration = {'LocationConstraint': S3_LOCATION})
        print(pprint.pprint(create_bucket))
except ClientError as e:
    print(e)

{'Location': 'http://hfelipini-covid19-project.s3.amazonaws.com/',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
                                      'date': 'Sat, 10 Dec 2022 20:36:08 GMT',
                                      'location': 'http://hfelipini-covid19-project.s3.amazonaws.com/',
                                      'server': 'AmazonS3',
                                      'x-amz-id-2': 'iD2rA4QIxwz9cx8piWg3NN9Y52ExBp/wFeOsqmypQwGZrwh2yQTl2iXR7aG+N1v5Tp9pU8wKiIcTaiWkYJI1LA==',
                                      'x-amz-request-id': 'P8JN1VAKQNR7TWSY'},
                      'HTTPStatusCode': 200,
                      'HostId': 'iD2rA4QIxwz9cx8piWg3NN9Y52ExBp/wFeOsqmypQwGZrwh2yQTl2iXR7aG+N1v5Tp9pU8wKiIcTaiWkYJI1LA==',
                      'RequestId': 'P8JN1VAKQNR7TWSY',
                      'RetryAttempts': 0}}
None


2st Step - Copy Dataset into the created Bucket
Dataset chosen: https://registry.opendata.aws/aws-covid19-lake/

In [47]:
InfosToCopy = pd.read_csv('dataInfosCopy.csv', delimiter=";", header=None)
#print(InfosToCopy.to_string())
InfosToCopy.head()

,0,1,2,3
0,covid19-lake,enigma-jhu/csv/Enigma-JHU.csv.gz,Enigma-JHU.csv.gz,enigma-jhu/csv/
1,covid19-lake,enigma-nytimes-data-in-usa/csv/us_county/us_co...,us_county.csv,enigma-nytimes-data-in-usa/csv/us_county/
2,covid19-lake,enigma-nytimes-data-in-usa/csv/us_states/us_st...,us_states.csv,enigma-nytimes-data-in-usa/csv/us_states/
3,covid19-lake,rearc-covid-19-testing-data/csv/states_daily/s...,states_daily.csv,rearc-covid-19-testing-data/csv/states_daily/
4,covid19-lake,rearc-covid-19-testing-data/csv/us-total-lates...,us.csv,rearc-covid-19-testing-data/csv/us-total-latest/


In [57]:
#InfosToCopy.iloc[0,2]
#len(InfosToCopy)

#for position in range(len(InfosToCopy)):
#    print(position)

In [58]:
s3 = boto3.resource('s3', 
                aws_access_key_id=AWS_KEY,
                aws_secret_access_key=AWS_SECRET)
                
for position in range(len(InfosToCopy)):
    s3.Bucket(S3_BUCKET_NAME).copy({"Bucket": InfosToCopy.iloc[position,0], "Key": InfosToCopy.iloc[position,1]}, InfosToCopy.iloc[position,2])
    print(position)
"""
s3.Bucket(S3_BUCKET_NAME).copy({"Bucket": 'covid19-lake', "Key": 'enigma-jhu/csv/Enigma-JHU.csv.gz'}, "Enigma-JHU.csv.gz")
s3.Bucket(S3_BUCKET_NAME).copy({"Bucket": 'covid19-lake', "Key": 'enigma-nytimes-data-in-usa/csv/us_county/us_county.csv'}, "us_county.csv")
s3.Bucket(S3_BUCKET_NAME).copy({"Bucket": 'covid19-lake', "Key": 'enigma-nytimes-data-in-usa/csv/us_states/us_states.csv'}, "us_states.csv")
s3.Bucket(S3_BUCKET_NAME).copy({"Bucket": 'covid19-lake', "Key": 'rearc-covid-19-testing-data/csv/states_daily/states_daily.csv'}, "states_daily.csv")
s3.Bucket(S3_BUCKET_NAME).copy({"Bucket": 'covid19-lake', "Key": 'rearc-covid-19-testing-data/csv/us-total-latest/us.csv'}, "us.csv")
s3.Bucket(S3_BUCKET_NAME).copy({"Bucket": 'covid19-lake', "Key": 'rearc-covid-19-testing-data/csv/us_daily/us_daily.csv'}, "us_daily.csv")
s3.Bucket(S3_BUCKET_NAME).copy({"Bucket": 'covid19-lake', "Key": 'rearc-usa-hospital-beds/json/usa-hospital-beds.geojson'}, "usa-hospital-beds.geojson")
s3.Bucket(S3_BUCKET_NAME).copy({"Bucket": 'covid19-lake', "Key": 'static-datasets/csv/countrycode/CountryCodeQS.csv'}, "CountryCodeQS.csv")
s3.Bucket(S3_BUCKET_NAME).copy({"Bucket": 'covid19-lake', "Key": 'static-datasets/csv/CountyPopulation/County_Population.csv'}, "County_Population.csv")
s3.Bucket(S3_BUCKET_NAME).copy({"Bucket": 'covid19-lake', "Key": 'static-datasets/csv/state-abv/states_abv.csv'}, "states_abv.csv")
"""

'\ns3.Bucket(S3_BUCKET_NAME).copy({"Bucket": \'covid19-lake\', "Key": \'enigma-jhu/csv/Enigma-JHU.csv.gz\'}, "Enigma-JHU.csv.gz")\ns3.Bucket(S3_BUCKET_NAME).copy({"Bucket": \'covid19-lake\', "Key": \'enigma-nytimes-data-in-usa/csv/us_county/us_county.csv\'}, "us_county.csv")\ns3.Bucket(S3_BUCKET_NAME).copy({"Bucket": \'covid19-lake\', "Key": \'enigma-nytimes-data-in-usa/csv/us_states/us_states.csv\'}, "us_states.csv")\ns3.Bucket(S3_BUCKET_NAME).copy({"Bucket": \'covid19-lake\', "Key": \'rearc-covid-19-testing-data/csv/states_daily/states_daily.csv\'}, "states_daily.csv")\ns3.Bucket(S3_BUCKET_NAME).copy({"Bucket": \'covid19-lake\', "Key": \'rearc-covid-19-testing-data/csv/us-total-latest/us.csv\'}, "us.csv")\ns3.Bucket(S3_BUCKET_NAME).copy({"Bucket": \'covid19-lake\', "Key": \'rearc-covid-19-testing-data/csv/us_daily/us_daily.csv\'}, "us_daily.csv")\ns3.Bucket(S3_BUCKET_NAME).copy({"Bucket": \'covid19-lake\', "Key": \'rearc-usa-hospital-beds/json/usa-hospital-beds.geojson\'}, "usa-hospi

https://github.com/oovk/dataengg-covid19-aws/blob/main/covid19_project.ipynb